# Setting

In [1]:
import pandas as pd
import re
import requests as rq
from bs4 import BeautifulSoup

## Function

In [10]:
def crawl(url, n, product_name):
    # 페이지 수
    pages = n*10  # n개까지 리뷰를 충분히 수집할 수 있도록 페이지 수 설정

    # 리뷰를 저장할 리스트 생성
    reviews = []
    ratings = []
    days = []

    # URL 재가공
    prod_code = url.split('products/')[-1].split('?')[0]
    urls = [
        f'https://www.coupang.com/vp/product/reviews?productId={prod_code}&page={page}&size=5&sortBy=DATE_DESC&ratings=&q=&viRoleCode=3&ratingSummary=true'
        for page in range(1, pages + 1)]

    # 접속 차단 우회
    headers = {
        "authority": "www.coupang.com",
        "method": "GET",
        "accept": "*/*",
        "accept-encoding": "gzip, deflate, br",
        "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.104 Whale/3.13.131.36 Safari/537.36",
        "sec-ch-ua-platform": "macOS",
        "sec-ch-ua-mobile": "?0",
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-origin",
        "cookie": "PCID=31489593180081104183684; _fbp=fb.1.1644931520418.1544640325; gd1=Y; X-CP-PT-locale=ko_KR; MARKETID=31489593180081104183684; sid=03ae1c0ed61946c19e760cf1a3d9317d808aca8b; overrideAbTestGroup=%5B%5D; x-coupang-origin-region=KOREA; x-coupang-accept-language=ko_KR;"
        }
    
    # 크롤링
    while len(reviews) < n:
        for url in urls:
            session = rq.Session()
            headers['referer'] = url
            response = session.get(url=url, headers=headers)
            html = response.text
            soup = BeautifulSoup(html, 'html.parser')

            # 리뷰 박스
            article_lenth = len(soup.select('article.sdp-review__article__list'))

            # 1~5번 리뷰 수집
            for idx in range(article_lenth):
                articles = soup.select('article.sdp-review__article__list')

                # 리뷰 본문
                review_content = articles[idx].select_one('div.sdp-review__article__list__review > div')
                if review_content == None:
                    continue   # 리뷰 본문이 없는 경우 다음 리뷰로 넘어감
                else:
                    reviews.append(re.sub('[\n\t]', '', review_content.text.strip()))

                    # 리뷰 본문이 있는 경우에만 별점, 리뷰작성일 수집
                    # 별점
                    rating = articles[idx].select_one('div.sdp-review__article__list__info__product-info__star-orange')
                    if rating == None:
                        ratings.append(0)
                    else:
                        ratings.append(int(rating.attrs['data-rating']))

                    # 리뷰작성일
                    day = articles[idx].select_one('div.sdp-review__article__list__info__product-info__reg-date')
                    days.append(day.text)

            if len(reviews) >= n:  # 리뷰 개수가 n 이상이면 루프 종료
                break

    # 크롤링 결과
    df = pd.DataFrame({'reviews': reviews, 'ratings': ratings, 'days': days})
    print(f'\n수집한 데이터는 다음과 같습니다. \n {df.head(4)}')

    df.to_csv(f"{product_name}.csv", encoding='utf-8-sig')

## Run

In [11]:
url1 = 'https://www.coupang.com/vp/products/21277951?itemId=83185355&vendorItemId=74185081497&src=1032001&spec=10305199&addtag=400&ctag=21277951&lptag=I83185355&itime=20240315115333&pageType=PRODUCT&pageValue=21277951&wPcid=17104712136363137870172&wRef=cr.shopping.naver.com&wTime=20240315115333&redirect=landing&mcid=96a64a71f4a84950a8e05f2294b21678&isAddedCart='
crawl(url = url1, n = 100, product_name = '게메즈 에낙 라면 과자 720g')


수집한 데이터는 다음과 같습니다. 
                                              reviews  ratings        days
0                    크기가 적당해 하나씩 먹기 좋아서 자주 구매하고 있습니다        5  2024.03.18
1  우연히 친구를 통해 처음 에낙을 접하고 너무 맛있어서바로 한박스를 주문했습니다.작은...        5  2024.03.16
2                                   저렴한 가격에 잘 구매했습니다        4  2024.03.16
3  편의점에 가서 사먹어보고 엄청 맛있더라 1개식 사면 가격 좀 비싸더라구요. 쿠팡에서...        5  2024.03.16


In [12]:
url2 = 'https://www.coupang.com/vp/products/7608804797?itemId=20146298818&vendorItemId=87239403818&pickType=COU_PICK&q=%EC%97%90%EC%96%B4%ED%8C%9F+%ED%94%84%EB%A1%9C&itemsCount=36&searchId=74335e87972a42f7b6c14e7e6f3c6cf6&rank=0&isAddedCart='
crawl(url = url2, n = 100, product_name = 'Apple 2023 에어팟 프로 2세대')


수집한 데이터는 다음과 같습니다. 
                                              reviews  ratings        days
0  아직 3월 다 가지도 않았는데 24년 3월 제품이라니 감사합니다제품 받자마자 기기 ...        5  2024.03.19
1  4년동안 사용하던 에어팟이 고장나서 새로 구매하게되었어요. 며칠되지 않았지만 아직까...        5  2024.03.18
2               배송이 정말정말 빨랐어요!제품도 새제품 그리고 상태도 너무 좋아요        5  2024.03.18
3  처음 노이즈캔슬링 접한 사람으로써 노캔 기능은 정말 행복이라고 느껴집니다 주변 소리...        5  2024.03.18


In [13]:
url3 = 'https://www.coupang.com/vp/products/111269023?itemId=335162862&vendorItemId=3822330082&pickType=COU_PICK&q=%EB%96%A1%EB%B3%B6%EC%9D%B4&itemsCount=36&searchId=c3c994381a9d432689ce7e991eaeddfc&rank=1&isAddedCart='
crawl(url = url3, n = 100, product_name = '풀무원 밀 누들 국물떡볶이 2인분 423.5g')


수집한 데이터는 다음과 같습니다. 
                                              reviews  ratings        days
0  누들국물떡볶이 양념도 맛잇고 밀떡도 길어서 아이가 신기해 하면서도 잘먹어여~풀무원음...        5  2024.03.19
1  매번 먹는 떡이 아니라 떡이 기니 아이들이 좋아하네요쿠팡 추천템이라 구매해봤는데 너...        5  2024.03.18
2  오뚜기 떡볶이에는 라면사리가 들어있는데 풀무원 떡볶이에는 없네요 그점이 좀 아쉬워요...        4  2024.03.18
3  배달음식보다 빨리되고 간편하고 더 맛있어요 추천합니다세번째 구매입니다 또 재구매의사...        5  2024.03.17
